# Vector Store as retriever to Memory in ConversationChain to store multiple chat histories.

This notebook goes over how to use a Vector Store as retriever to Memory class with a `ConversationChain` to store multiple chat histories based on metadata. 

We will add the [VectorStoreRetrieverMemory](https://api.python.langchain.com/en/latest/memory/langchain.memory.vectorstore.VectorStoreRetrieverMemory.html#langchain.memory.vectorstore.VectorStoreRetrieverMemory) class, using the [PGVector](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.pgvector.PGVector.html#langchain.vectorstores.pgvector.PGVector) vector store.

In [1]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import VectorStoreRetrieverMemory
from langchain.vectorstores.pgvector import PGVector
from langchain.embeddings.openai import OpenAIEmbeddings

We initialize our `VectorStore` and some metadata to work with.

In [2]:
vectorstore = PGVector(
    collection_name="<COLLECTION_NAME>",
    connection_string="<CONNECTION_STRING>",
    embedding_function=OpenAIEmbeddings()
)

metadata = {
    "user_name": "John",
}

Then, we create our `ConversationChain` using the `VectorStoreRetrieverMemory`. Notice that in order to store and retrieve the chat history corresponding to the metadata, we need to pass it to the `VectorStoreRetrieverMemory` and `VectorStore` search_kwargs filter.

In [3]:
memory = VectorStoreRetrieverMemory(
    retriever=vectorstore.as_retriever(
        search_kwargs=dict(
            filter=metadata,
        )
    ),
    metadata=metadata
)

chain = ConversationChain(
    llm=OpenAI(temperature=0),
    memory=memory,
)

In [4]:
chain.run(input="Hi, my name is John.")
chain.run(input="What is my name?")

' Your name is John.'

Using new metadata, we can use the chain without the chat histories getting mixed up.

In [5]:
new_metadata = {
    "user_name": "Doe",
}

memory = VectorStoreRetrieverMemory(
    retriever=vectorstore.as_retriever(
        search_kwargs=dict(
            filter=new_metadata,
        )
    ),
    metadata=new_metadata
)

chain = ConversationChain(
    llm=OpenAI(temperature=0),
    memory=memory,
)


First, we check that the chain does not know our name.

In [6]:
chain.run(input="What is my name?")

' I don't know what your name is. Can you tell me your name? '

In [7]:
chain.run(input="Hi, my name is Doe.")
chain.run(input="What is my name?")

' Your name is Doe.'